# SoccerNet - Deep Hough Transform - training own model

Instructions: DO NOT RUN ALL CELLS AUTOMATICALLY. Kindly run the cells for the first section manually, and then RUN AFTER ALL CELLS AUTOMATICALLY

## Get Deep Hough Transform repository and adapt (Manual)

In [ ]:
!git clone https://github.com/Hanqer/deep-hough-transform.git

In [ ]:
# fix deep-hough-transform/data/prepare_data_NKL.py
# Replace lines 70 - 103 with:
'''
            with open(join(label_path, label_file)) as f:
                data = f.readlines()[0].split(' ')
                nums = int(data[0])
                # stastic[nums] += 1
                total_nums += nums
                
                if int(nums) == 0:
                    print("Warning: image has no semantic line : %s" % (filename))

                for i in range(nums):
                    y1, x1 = check(int(data[i*4+2]), int(data[i*4+1]), H, W)
                    y2, x2 = check(int(data[i*4+4]), int(data[i*4+3]), H, W)
                    if y1 == y2:
                      y2 += 1
                      y1 -= 1
                    if x1 == x2:
                      x2 += 1
                      x1 -= 1
                    line = Line([y1, x1, y2, x2])
                    angle = line.angle()
                    angle_stastic[int((angle / np.pi + 0.5) * 180)] += 1
                    total_lines += 1
                    line.rescale(scale_H, scale_W)
                    lines.append(line)
            
            annotation = LineAnnotation(size=[args.fixsize, args.fixsize], lines=lines)
        else:
            im = cv2.flip(im, 1)
            filename = filename + '_flip'
            lines = []
            with open(join(label_path, label_file)) as f:
                data = f.readlines()[0].split(' ')
                for i in range(int(data[0])):
                    y1, x1 = check(int(data[i*4+2]), W-1-int(data[i*4+1]), H, W)
                    y2, x2 = check(int(data[i*4+4]), W-1-int(data[i*4+3]), H, W)
                    if y1 == y2:
                      y2 += 1
                      y1 -= 1
                    if x1 == x2:
                      x2 += 1
                      x1 -= 1
                    line = Line([y1, x1, y2, x2])
                    line.rescale(scale_H, scale_W)
                    lines.append(line)
            
            annotation = LineAnnotation(size=[args.fixsize, args.fixsize], lines=lines)
'''

In [ ]:
# fix deep-hough-transform/config.yml
# replace the lines 2 - 13 with:
'''
    DIR: "data/training/"
    VAL_DIR: "data/training/"
    TEST_DIR: "data/soccernet_data_test"
    LABEL_FILE: "data/training/train_soccernet.txt"
    VAL_LABEL_FILE: "data/training/valid_soccernet.txt"
    TEST_LABEL_FILE: "data/training/test_soccernet.txt"
'''
# SET THE EPOCHS TO 3 OR AS MANY AS YOU WANT

In [ ]:
# fix deep-hough-transform/train.py
# Line: 34
# CONFIGS = yaml.load(open(args.config), Loader=yaml.Loader)

## Get SoccerNet data and adapt

In [ ]:
!pip install SoccerNet

In [ ]:
from SoccerNet.Downloader import SoccerNetDownloader
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory="SoccerNet/")
mySoccerNetDownloader.downloadDataTask(task="calibration", split=["train","valid","test","challenge"])

In [ ]:
# Prepare Soccernet data
%cd SoccerNet/calibration/

In [ ]:
!unzip train.zip

In [ ]:
!unzip test.zip

In [ ]:
!unzip valid.zip

In [ ]:
%cd train/
%mkdir soccernet_data_train

In [ ]:
import math 
for x in range(14513):
  lineCount = 0
  x_coords = []
  y_coords = []
  foundCircle = False
  if x >= 10000:
    prefix = ""
  if x < 10000:
    prefix = "0"
  if x < 1000:
    prefix = "00"
  if x < 100:
    prefix = "000"
  if x < 10:
    prefix = "0000"
  with open(prefix + str(x) + '.txt', 'w') as fw:
    with open(prefix + str(x) + '.json', 'r') as fr:
      for line in fr.readlines():
        if 'Circle' not in line and '[' in line:
          foundCircle = False
        if foundCircle == True:
          continue
        if 'Circle' in line:
          foundCircle = True
          continue
        if '"x":' in line:
          x_coords.append(int(math.floor(float(line.split(': ')[1].split(',')[0]) * 960)))
        elif '"y":' in line:
          y_coords.append(int(math.floor(float(line.split(': ')[1].split(',')[0]) * 960)))
      lineCount = len(x_coords) // 2
      fw.write(str(lineCount))
      for i in range(len(x_coords)):
        fw.write(' ' + str(x_coords[i]) + ' ' + str(y_coords[i]))
      fr.close()
    fw.close()

In [ ]:
%mv *.jpg soccernet_data_train/
%mv *.txt soccernet_data_train/
%cp -r soccernet_data_train ../../../deep-hough-transform/data/

In [ ]:
%cd ../valid/
%mkdir soccernet_data_valid

In [ ]:
import math 
for x in range(2796):
  lineCount = 0
  x_coords = []
  y_coords = []
  foundCircle = False
  if x >= 10000:
    prefix = ""
  if x < 10000:
    prefix = "0"
  if x < 1000:
    prefix = "00"
  if x < 100:
    prefix = "000"
  if x < 10:
    prefix = "0000"
  with open(prefix + str(x) + '.txt', 'w') as fw:
    with open(prefix + str(x) + '.json', 'r') as fr:
      for line in fr.readlines():
        if 'Circle' not in line and '[' in line:
          foundCircle = False
        if foundCircle == True:
          continue
        if 'Circle' in line:
          foundCircle = True
          continue
        if '"x":' in line:
          x_coords.append(int(math.floor(float(line.split(': ')[1].split(',')[0]) * 960)))
        elif '"y":' in line:
          y_coords.append(int(math.floor(float(line.split(': ')[1].split(',')[0]) * 960)))
      lineCount = len(x_coords) // 2
      fw.write(str(lineCount))
      for i in range(len(x_coords)):
        fw.write(' ' + str(x_coords[i]) + ' ' + str(y_coords[i]))
      fr.close()
    fw.close()

In [ ]:
%mv *.jpg soccernet_data_valid/
%mv *.txt soccernet_data_valid/
%cp -r soccernet_data_valid ../../../deep-hough-transform/data/

In [ ]:
%cd ../test/
%mkdir soccernet_data_test

In [ ]:
import math 
for x in range(2719):
  lineCount = 0
  x_coords = []
  y_coords = []
  foundCircle = False
  if x >= 10000:
    prefix = ""
  if x < 10000:
    prefix = "0"
  if x < 1000:
    prefix = "00"
  if x < 100:
    prefix = "000"
  if x < 10:
    prefix = "0000"
  with open(prefix + str(x) + '.txt', 'w') as fw:
    with open(prefix + str(x) + '.json', 'r') as fr:
      for line in fr.readlines():
        if 'Circle' not in line and '[' in line:
          foundCircle = False
        if foundCircle == True:
          continue
        if 'Circle' in line:
          foundCircle = True
          continue
        if '"x":' in line:
          x_coords.append(int(math.floor(float(line.split(': ')[1].split(',')[0]) * 960)))
        elif '"y":' in line:
          y_coords.append(int(math.floor(float(line.split(': ')[1].split(',')[0]) * 960)))
      lineCount = len(x_coords) // 2
      fw.write(str(lineCount))
      for i in range(len(x_coords)):
        fw.write(' ' + str(x_coords[i]) + ' ' + str(y_coords[i]))
      fr.close()
    fw.close()

In [ ]:
%mv *.jpg soccernet_data_test/
%mv *.txt soccernet_data_test/
%cp -r soccernet_data_test ../../../deep-hough-transform/data/

In [ ]:
%cd ../../../deep-hough-transform/

## Install Deep Hough Transform dependency

In [ ]:
%cd model/_cdht

In [ ]:
!python setup.py build

In [ ]:
!python setup.py install --user

In [ ]:
%cd ../../

## Prepare the training data and validation data

In [ ]:
!pip install tensorboardX

In [ ]:
!pip install POT

In [ ]:
!pip install Ninja

In [ ]:
# Prepare training data
!python data/prepare_data_NKL.py --root './data/soccernet_data_train' --label './data/soccernet_data_train' --save-dir './data/training/soccernet_train_resize_100_100' --fixsize 400

In [ ]:
# Prepare validation data
!python data/prepare_data_NKL.py --root './data/soccernet_data_valid' --label './data/soccernet_data_valid' --save-dir './data/training/soccernet_valid_resize_100_100' --fixsize 400

In [ ]:
# NO NEED FOR THIS
# Prepare testing data
# !python data/prepare_data_NKL.py --root './data/soccernet_data_test' --label './data/soccernet_data_test' --save-dir './data/training/soccernet_test_resize_100_100' --fixsize 400

In [ ]:
# Prepare txt files
%cd data/training/

In [ ]:
with open('train_soccernet.txt', 'w') as f:
  for i in range(0, 14513):
    if i >= 10000:
      prefix = ""
    if i < 10000:
      prefix = "0"
    if i < 1000:
      prefix = "00"
    if i < 100:
      prefix = "000"
    if i < 10:
      prefix = "0000"
    f.write('soccernet_train_resize_100_100/' + prefix + str(i) + '\n')
    f.write('soccernet_train_resize_100_100/' + prefix + str(i) + '_flip' + '\n')
  f.close()

In [ ]:
with open('valid_soccernet.txt', 'w') as f:
  for i in range(0, 2796):
    if i >= 10000:
      prefix = ""
    if i < 10000:
      prefix = "0"
    if i < 1000:
      prefix = "00"
    if i < 100:
      prefix = "000"
    if i < 10:
      prefix = "0000"
    f.write('soccernet_valid_resize_100_100/' + prefix + str(i) + '\n')
    # Not including flips in this
    # f.write('soccernet_valid_resize_100_100/' + prefix + str(i) + '_flip' + '\n')
  f.close()

In [ ]:
with open('test_soccernet.txt', 'w') as f:
  for i in range(0, 2719):
    if i >= 10000:
      prefix = ""
    if i < 10000:
      prefix = "0"
    if i < 1000:
      prefix = "00"
    if i < 100:
      prefix = "000"
    if i < 10:
      prefix = "0000"
    f.write(prefix + str(i) + '\n')

In [ ]:
%cd ../../

## Train the model (Download from runtime)

In [ ]:
!python train.py

In [ ]:
!zip result.zip result

In [ ]:
# Download result.zip and the trained model can be found as model_best.pth under the reproduce directory